In [19]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.base import BaseEstimator, TransformerMixin
import fasttext
import pandas as pd
from scipy import spatial
from scipy.spatial import distance
from resources.basicIO import InputOutput as IO
from resources.tokTT import CommentTokenizer
from sklearn import svm
from sklearn import metrics
from sklearn.linear_model import SGDClassifier
#from sklearn.linear_model import LassoLars     
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfTransformer


In [20]:
class FastTextTransformer(BaseEstimator, TransformerMixin):
    """ Convert texts into their mean fastText vectors """

    def __init__(self, model):
        self.model = model

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return np.stack([np.mean([self.model[w] for w in text.split()], 0) for text in X])

In [21]:
def classify(small_model,predictor,lines,Y):
    classifier = make_pipeline(
        FastTextTransformer(model=small_model),
        predictor
    ).fit(
        lines,
        Y
    )
    return classifier


### Load Models

In [22]:
# load models
model_N_2 = fasttext.load_model('models/ft_unsupervised_N_2.bin')

In [23]:
# Load seed set and tokenize
seed_set = CommentTokenizer.cleaned("datasets/seed_set.txt")
# Load seed Labels
Y = IO.load_nums("datasets/seed_set_labels.txt")

# Load expanded seed set
seed_set_expanded_N_2 = CommentTokenizer.cleaned("datasets_post/batch_N_2.txt")
Y_N_2 = IO.load_nums("datasets_post/batch_labels_N_2.txt")


In [24]:
# Load testing set
testing_text = IO.load_csv_col('datasets/random_sample.csv', 'comment')
testing_text = testing_text[350:700]
testing_text = [CommentTokenizer.tokenize(x) for x in testing_text]

testing_text_labels = IO.load_csv_col('datasets/random_sample.csv', 'label')
testing_text_labels = list(map(int, testing_text_labels[350:700]))

training_text = IO.load_csv_col('datasets/random_sample.csv', 'comment')
training_text = training_text[0:350]
training_text = [CommentTokenizer.tokenize(x) for x in training_text]

training_text_labels = IO.load_csv_col('datasets/random_sample.csv', 'label')
training_text_labels = list(map(int, training_text_labels[0:350]))


In [25]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline(
    [
        ("vect", CountVectorizer()),
        ("tfidf", TfidfTransformer()),
        ("clf", LogisticRegression()),
    ]
)

model = pipeline.fit(training_text, training_text_labels)


In [26]:
y_pred = model.predict(testing_text)
accuracy = metrics.accuracy_score(testing_text_labels, y_pred)
print(accuracy)

0.6857142857142857


In [27]:
def transform(model, X):
    return np.stack([np.mean([model[w] for w in text.split()], 0) for text in X])

X = ["I am in support of modiji he is right"]
transform(model_N_2,X)

array([[ 0.26113525,  0.04918763,  0.01847435, -0.20160277,  0.07977112,
         0.54330593, -0.1013101 ,  0.11043589,  0.14657088,  0.72111475,
         0.06386383,  0.31313893, -0.7443851 , -0.04762344, -0.107664  ,
         0.26124305,  0.3631335 , -0.06800381,  0.23122048, -0.0240441 ,
         0.06630781, -0.07395054,  0.03761048, -0.13507582, -0.18287705,
         0.12068001,  0.22809502, -0.05310457,  0.02926822,  0.04191326,
         0.28621927,  0.10182662,  0.00289312, -0.15193664,  0.47005123,
         0.16636842,  0.5011269 , -0.38512352, -0.54478955,  0.32894796,
        -0.5748296 ,  0.05187561, -0.02580795,  0.00782517,  0.05645113,
        -0.39001477,  0.13602103,  0.23150386,  0.2136382 ,  0.02123973,
        -0.4922437 , -0.11598277,  0.09155013,  0.18206777,  0.00719333,
        -0.16943865, -0.5857889 ,  0.24411495, -0.19438696,  0.5449885 ,
        -0.16829127, -0.07384431,  0.08972307, -0.2555257 , -0.18613236,
         0.12384824, -0.2101133 ,  0.09150001, -0.2

## Models

### LR

In [28]:
# classification
LR_ss_N_2 = classify(model_N_2, LogisticRegression(
    random_state=1), seed_set, Y)
LR_es_N_2 = classify(model_N_2, LogisticRegression(
    random_state=1), seed_set_expanded_N_2, Y_N_2)


### SVM

In [29]:
SVM_ss_N_2 = classify(model_N_2, svm.SVC(), seed_set, Y)
SVM_es_N_2 = classify(model_N_2, svm.SVC(), seed_set_expanded_N_2, Y_N_2)


### SGD

In [30]:
SGD_ss_N_2 = classify(model_N_2, SGDClassifier(
    loss="hinge", penalty="l1"), seed_set, Y)
SGD_es_N_2 = classify(model_N_2, SGDClassifier(
    loss="hinge", penalty="l1"), seed_set_expanded_N_2, Y_N_2)


### LDA

In [31]:
LDA_ss_N_2 = classify(model_N_2, LinearDiscriminantAnalysis(), seed_set, Y)
LDA_es_N_2 = classify(model_N_2, LinearDiscriminantAnalysis(),seed_set_expanded_N_2, Y_N_2)

### Decision Tree

In [32]:
DT_ss_N_2 = classify(model_N_2, DecisionTreeClassifier(), seed_set, Y)
DT_es_N_2 = classify(model_N_2, DecisionTreeClassifier(),
                      seed_set_expanded_N_2, Y_N_2)

### Gaussian NB

In [33]:
GNB_ss_N_2 = classify(model_N_2, GaussianNB(), seed_set, Y)
GNB_es_N_2 = classify(model_N_2, GaussianNB(),
                      seed_set_expanded_N_2, Y_N_2)

### Random Forest

In [34]:
RF_ss_N_2 = classify(model_N_2, RandomForestClassifier(
    n_estimators=50, random_state=1), seed_set, Y)
RF_es_N_2 = classify(model_N_2, RandomForestClassifier(
    n_estimators=50, random_state=1),seed_set_expanded_N_2, Y_N_2)


### Utility for all models

In [35]:
models = []
models.append(('Gaussian NB Seed Set N=2', GNB_ss_N_2))
models.append(('Gaussian NB Expanded Set N=2', GNB_es_N_2))
models.append(('LR Expanded Set N=2', LR_es_N_2))
models.append(('SVM Expanded Set N=2', SVM_es_N_2))
models.append(('SGD Expanded Set N=2', SGD_es_N_2))
models.append(('LDA Expanded Set N=2', LDA_es_N_2))
models.append(('Decision Trees Expanded Set N=2', DT_es_N_2))
models.append(('Random Forest Expanded Set N=2', RF_es_N_2))


### Print output to file

In [36]:
file = open('results/output.txt', 'w+')
file.close()

outfile = open("results/output.txt", "a")
for i, v in models:
    print(i)
    accuracy = metrics.accuracy_score(testing_text_labels, v.predict(testing_text))
    confusion_matrix = metrics.confusion_matrix(testing_text_labels, v.predict(testing_text))
    print('========= {} Model Test Results ==========='.format(i), file=outfile) 
    print(' ',file=outfile)
    print("Model Accuracy:" "\n", accuracy, file=outfile)
    print(' ', file=outfile)
    print("Confusion matrix:" "\n", confusion_matrix, file=outfile)
    print(' ', file=outfile)
outfile.close()


Gaussian NB Seed Set N=2
Gaussian NB Expanded Set N=2
LR Expanded Set N=2
SVM Expanded Set N=2
SGD Expanded Set N=2
LDA Expanded Set N=2
Decision Trees Expanded Set N=2
Random Forest Expanded Set N=2
